**PIP Installs required for the project**

In [1]:
!pip install -U datasets
!pip install fsspec
!pip install transformers datasets evaluate sentence-transformers faiss-cpu streamlit
!pip install -U transformers
!pip install rouge_score
!pip install -U huggingface_hub
!pip install -U transformers datasets evaluate rouge_score bert_score textstat

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 79.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 132.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 123.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 119.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.

**Mounting Google Drive**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Loading dataset from Drive**

In [3]:
import pandas as pd

file_path = '/content/drive/MyDrive/mimic-iv-bhc.csv'

df = pd.read_csv(file_path)

**Preprocessing the Dataset**

In [4]:
import pandas as pd
import re
import html

def preprocess_row(text: str) -> str:
    if not isinstance(text, str):
        return ""

    text = html.unescape(text)

    text = text.replace("’", "'").replace("‘", "'")\
               .replace("“", '"').replace("”", '"')\
               .replace("–", "-").replace("—", "-")

    text = re.sub(r"[^\x00-\x7F]+", " ", text)

    text = re.sub(r"\s+", " ", text).strip()

    text = text.lower()

    return text

df["input"] = df["input"].apply(preprocess_row)
df["target"] = df["target"].apply(preprocess_row)


df[["input", "target"]].sample(3).to_dict(orient="records")

[{'input': "<sex> m <service> medicine <allergies> no known allergies / adverse drug reactions <attending> ___ <chief complaint> dyspnea, fever, tachycardia <major surgical or invasive procedure> none <history of present illness> ___ with h/o htn, chf, cva s/p hemiplegia, dementia, seizure disorder on ___ who was biba from nursing facility for reported shortness of breath and fever concerning for pneumonia. unclear timeline, and no known medications given pta. patient is dnr/dni but ok to transfer. ems noted patient had soft bp 99/63 and was tachycardic to 130s. had o2 86% on 2l but improved to >95% with high-flow. patient unable to provide full ros at this time. molst does not indicate wishes on bipap/cpap. in the ed: interview limited by ams. initial vital signs notable for t 98.1 hr 136 bp 99/63 rr 22 o2 sat 97% on 5l nc. patient triggered for hr>130 with dyspnea. repeat t up to 104.8, received vanc, zosyn and tylenol, with subsequent t down to 99.8. he had an increasing oxygen requ

**Splitting Data into Training and Validation**

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset

df = df.dropna(subset=["input", "target"])

subset_df = df.sample(frac=0.4, random_state=42).reset_index(drop=True)

train_df, val_df = train_test_split(subset_df, test_size=0.1, random_state=42)
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)


**Checking Huggingface Login**

In [6]:
from huggingface_hub import whoami

try:
    user_info = whoami()
    print("Logged in as:", user_info["name"])
except Exception as e:
    print("Not logged in or token expired:", e)

Logged in as: Vidit202


**Generating summaries before finetuning**

In [7]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

def generate_predictions(dataset, model_name, num_samples=10, max_input_len=512, max_output_len=128):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

    preds = []
    refs = []

    samples = dataset.select(range(num_samples))

    for row in samples:
        inputs = tokenizer(row["input"], return_tensors="pt", truncation=True, padding=True, max_length=max_input_len)
        outputs = model.generate(**inputs, max_length=max_output_len, num_beams=4)
        pred = tokenizer.decode(outputs[0], skip_special_tokens=True)
        preds.append(pred)
        refs.append(row["target"])

    return preds, refs


In [8]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from evaluate import load
import pandas as pd
import numpy as np

rouge = load("rouge")
bleu = load("bleu")
bertscore = load("bertscore")

def evaluate_all_metrics(preds, refs):
    scores = {}

    rouge_result = rouge.compute(predictions=preds, references=refs)
    scores.update(rouge_result)

    bleu_result = bleu.compute(predictions=preds, references=[[r] for r in refs])
    scores['bleu'] = bleu_result['bleu']

    bert_result = bertscore.compute(predictions=preds, references=refs, lang="en")
    scores["bertscore_precision"] = sum(bert_result["precision"]) / len(bert_result["precision"])
    scores["bertscore_recall"] = sum(bert_result["recall"]) / len(bert_result["recall"])
    scores["bertscore_f1"] = sum(bert_result["f1"]) / len(bert_result["f1"])
    return scores

In [9]:
t5_preds, refs = generate_predictions(val_dataset, "t5-small", num_samples=10)
bart_preds, _ = generate_predictions(val_dataset, "facebook/bart-base", num_samples=10)
pegasus_preds, _ = generate_predictions(val_dataset, "google/pegasus-pubmed", num_samples=10)

metrics = {
    "T5": evaluate_all_metrics(t5_preds, refs),
    "BART": evaluate_all_metrics(bart_preds, refs),
    "Pegasus": evaluate_all_metrics(pegasus_preds, refs)
}

import pandas as pd
import numpy as np

df_metrics = pd.DataFrame(metrics).T.round(4)
df_metrics.replace({np.nan: "N/A"}, inplace=True)
df_metrics


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


,rouge1,rouge2,rougeL,rougeLsum,bleu,bertscore_precision,bertscore_recall,bertscore_f1
T5,0.1256,0.0347,0.0821,0.0821,0.0006,0.8321,0.7927,0.8118
BART,0.1711,0.0561,0.1081,0.1090,0.0101,0.8217,0.8026,0.8119
Pegasus,0.1509,0.0233,0.1000,0.1014,0.0007,0.8057,0.7877,0.7964


**FineTuning the 3 Models using our dataset**

In [ ]:
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq
)
from huggingface_hub import HfApi, login
import os

def run_training(model_name: str, hf_repo_id: str, run_name: str, train_dataset, val_dataset , isCheckpoint):
    HfApi().create_repo(
        repo_id=hf_repo_id,
        private=False,
        exist_ok=True
    )

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

    def preprocess_function(examples):
        inputs = [str(x) for x in examples["input"]]
        targets = [str(x) for x in examples["target"]]
        model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
        labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")["input_ids"]
        model_inputs["labels"] = labels
        return model_inputs

    tokenized_train = train_dataset.map(preprocess_function, batched=True, remove_columns=train_dataset.column_names)
    tokenized_val = val_dataset.map(preprocess_function, batched=True, remove_columns=val_dataset.column_names)

    data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

    training_args = Seq2SeqTrainingArguments(
        output_dir="./results_bart",
        per_device_train_batch_size=8,
        num_train_epochs=3,
        fp16=True,
        eval_strategy="epoch",
        predict_with_generate=True,
        save_strategy="steps",
        save_steps=1000,
        save_total_limit=2,
        logging_steps=200,
        report_to="wandb",
        run_name=run_name
    )

    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_val,
        tokenizer=tokenizer,
        data_collator=data_collator
    )

    if isCheckpoint :
      trainer.train(resume_from_checkpoint=True)
    else :
      trainer.train()


    model.save_pretrained(hf_repo_id, push_to_hub=True)
    tokenizer.save_pretrained(hf_repo_id, push_to_hub=True)

    print(f"✅ Model pushed publicly to: https://huggingface.co/{hf_repo_id}")


**Finetuning T5-Small**

In [ ]:
run_training(
    model_name="t5-small",
    hf_repo_id="Vidit202/t5-mimic-summary",
    run_name="t5-mimic-wandb",
    train_dataset=train_dataset,
    val_dataset=val_dataset
)

Map:   0%|          | 0/97211 [00:00<?, ? examples/s]

Map:   0%|          | 0/10802 [00:00<?, ? examples/s]

/tmp/ipython-input-361283414.py:49: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].
wandb: Currently logged in as: viditgn-21102 (viditgn-21102-northeastern-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
2,2.031700,1.884147
3,1.975100,1.839981
4,1.946700,1.827724


Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

...

  .../t5-mimic-summary/model.safetensors:   0%|          |  552kB /  242MB            

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  Vidit202/t5-mimic-summary/spiece.model: 100%|##########|  792kB /  792kB            

✅ Model pushed publicly to: https://huggingface.co/Vidit202/t5-mimic-summary


**Finetuning Bart-Base**

In [ ]:
run_training(
    model_name="facebook/bart-base",
    hf_repo_id="Vidit202/bart-mimic-summary",
    run_name="bart-mimic-wandb",
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    isCheckpoint=False
)

Map:   0%|          | 0/97211 [00:00<?, ? examples/s]

Map:   0%|          | 0/10802 [00:00<?, ? examples/s]

/tmp/ipython-input-2165548172.py:49: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Epoch,Training Loss,Validation Loss
1,1.969200,1.852442
2,1.793600,1.757909
3,1.726700,1.712669
4,1.659700,1.698678


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3852: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...art-mimic-summary/model.safetensors:   0%|          |  131kB /  558MB            

✅ Model pushed publicly to: https://huggingface.co/Vidit202/bart-mimic-summary


**Finetuning Pegasus-Pubmed**

In [ ]:
run_training(
    model_name="google/pegasus-pubmed",
    hf_repo_id="Vidit202/pegasus-pubmed-summary",
    run_name="pegasus-wandb",
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    isCheckpoint=False
)

tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

...

  ...us-pubmed-summary/model.safetensors:   0%|          | 4.39MB / 2.28GB            

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...pegasus-pubmed-summary/spiece.model:  90%|########9 | 1.71MB / 1.91MB            

✅ Model pushed publicly to: https://huggingface.co/Vidit202/pegasus-pubmed-summary


**Using models to generate summary**

In [10]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM
)
device = "cuda" if torch.cuda.is_available() else "cpu"

model_names = {
    "t5": "Vidit202/t5-mimic-summary",
    "bart": "Vidit202/bart-mimic-summary",
    "pegasus": "Vidit202/pegasus-pubmed-summary"
}

tokenizers = {k: AutoTokenizer.from_pretrained(v) for k, v in model_names.items()}
models = {k: AutoModelForSeq2SeqLM.from_pretrained(v).to(device) for k, v in model_names.items()}


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

...

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/292 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/275 [00:00<?, ?B/s]

In [ ]:
def generate_summary(model, tokenizer, text, max_input_len=512, max_output_len=128):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=max_input_len).to(device)
    outputs = model.generate(**inputs, max_length=max_output_len)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
def combined_summarizer(text):
    summaries = []
    for name in ["t5", "bart", "pegasus"]:
        summary = generate_summary(models[name], tokenizers[name], text)
        summaries.append(summary)

    merged_summary_input = " ".join(summaries)

    final_summary = generate_summary(models["t5"], tokenizers["t5"], merged_summary_input)

    return {
        "T5": summaries[0],
        "BART": summaries[1],
        "Pegasus": summaries[2],
        "Combined": final_summary
    }


**Evaluation Metrics**

In [ ]:
val_df = val_dataset.to_pandas()

results = []

for idx, row in val_df.iterrows():
    input_text = row["input"]
    reference = row["target"]

    outputs = combined_summarizer(input_text)

    results.append({
        "input": input_text,
        "reference": reference,
        "t5": outputs["T5"],
        "bart": outputs["BART"],
        "pegasus": outputs["Pegasus"],
        "combined": outputs["Combined"]
    })

    if idx >= 10:
        break


In [ ]:
import evaluate

rouge = evaluate.load("rouge")
bleu = evaluate.load("bleu")
bertscore = evaluate.load("bertscore")

In [ ]:
def evaluate_all_metrics(preds, refs):
    scores = {}

    rouge_result = rouge.compute(predictions=preds, references=refs)
    scores.update(rouge_result)

    bleu_result = bleu.compute(predictions=preds, references=[[r] for r in refs])
    scores['bleu'] = bleu_result['bleu']

    bert_result = bertscore.compute(predictions=preds, references=refs, lang="en")
    scores["bertscore_precision"] = sum(bert_result["precision"]) / len(bert_result["precision"])
    scores["bertscore_recall"] = sum(bert_result["recall"]) / len(bert_result["recall"])
    scores["bertscore_f1"] = sum(bert_result["f1"]) / len(bert_result["f1"])
    return scores


In [ ]:
t5_preds = [r["t5"] for r in results]
bart_preds = [r["bart"] for r in results]
pegasus_preds = [r["pegasus"] for r in results]
combined_preds = [r["combined"] for r in results]
refs = [r["reference"] for r in results]

metrics = {
    "T5": evaluate_all_metrics(t5_preds, refs),
    "BART": evaluate_all_metrics(bart_preds, refs),
    "Pegasus": evaluate_all_metrics(pegasus_preds, refs),
    "Combined": evaluate_all_metrics(combined_preds, refs)
}


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import pandas as pd
import numpy as np

df_metrics = pd.DataFrame(metrics).T.round(4)
df_metrics.replace({np.nan: "N/A"}, inplace=True)
df_metrics


,rouge1,rouge2,rougeL,rougeLsum,bleu,bertscore_precision,bertscore_recall,bertscore_f1
T5,0.2432,0.1151,0.1841,0.1820,0.0058,0.8758,0.8152,0.8441
BART,0.2889,0.1472,0.2080,0.2068,0.0171,0.8771,0.8273,0.8513
Pegasus,0.2812,0.1285,0.2070,0.2049,0.0176,0.8676,0.8231,0.8445
Combined,0.2197,0.1071,0.1657,0.1677,0.0046,0.8606,0.8073,0.8328


**Custom Dataset for RAG**

In [ ]:
medical_terms = {
    "hypertension": "high blood pressure",
    "hypotension": "low blood pressure",
    "myocardial infarction": "heart attack",
    "cerebrovascular accident": "stroke",
    "dyspnea": "shortness of breath",
    "analgesic": "painkiller",
    "edema": "swelling",
    "febrile": "feverish",
    "gastritis": "stomach inflammation",
    "neoplasm": "tumor",
    "hyperlipidemia": "high cholesterol",
    "renal failure": "kidney failure",
    "hematemesis": "vomiting blood",
    "hematuria": "blood in urine",
    "hematochezia": "blood in stool",
    "tachycardia": "fast heartbeat",
    "bradycardia": "slow heartbeat",
    "arrhythmia": "irregular heartbeat",
    "diaphoresis": "sweating",
    "syncope": "fainting",
    "nausea": "feeling like vomiting",
    "vomitus": "vomit",
    "anemia": "low red blood cells",
    "leukocytosis": "high white blood cells",
    "thrombocytopenia": "low platelets",
    "polyuria": "frequent urination",
    "nocturia": "urinating at night",
    "dysuria": "painful urination",
    "incontinence": "loss of bladder control",
    "hepatomegaly": "enlarged liver",
    "splenomegaly": "enlarged spleen",
    "hepatitis": "liver inflammation",
    "cirrhosis": "liver scarring",
    "osteoporosis": "weak bones",
    "osteoarthritis": "joint wear and tear",
    "rheumatoid arthritis": "joint inflammation",
    "embolism": "blood clot",
    "thrombosis": "blood clot formation",
    "ischemia": "lack of blood flow",
    "necrosis": "tissue death",
    "cyanosis": "bluish skin",
    "jaundice": "yellow skin",
    "pruritus": "itching",
    "urticaria": "hives",
    "eczema": "skin inflammation",
    "dermatitis": "skin irritation",
    "alopecia": "hair loss",
    "melena": "black stool",
    "hemoptysis": "coughing up blood",
    "pleurisy": "lung lining inflammation",
    "pneumonia": "lung infection",
    "bronchitis": "airway inflammation",
    "asthma": "narrowed airways",
    "copd": "chronic lung disease",
    "emphysema": "damaged lung sacs",
    "hypoxia": "low oxygen",
    "apnea": "not breathing",
    "dyspnea on exertion": "shortness of breath with activity",
    "orthopnea": "breathing difficulty when lying down",
    "cyanotic": "blue-colored skin",
    "tachypnea": "fast breathing",
    "bradypnea": "slow breathing",
    "sepsis": "body-wide infection",
    "bacteremia": "bacteria in the blood",
    "pyelonephritis": "kidney infection",
    "cystitis": "bladder infection",
    "nephrolithiasis": "kidney stones",
    "cholelithiasis": "gallstones",
    "cholecystitis": "gallbladder inflammation",
    "pancreatitis": "pancreas inflammation",
    "appendicitis": "appendix inflammation",
    "diverticulitis": "colon pouch inflammation",
    "gastroenteritis": "stomach flu",
    "colitis": "colon inflammation",
    "constipation": "hard stool",
    "diarrhea": "loose stool",
    "abdominal distension": "bloated belly",
    "ascites": "fluid in belly",
    "anorexia": "loss of appetite",
    "cachexia": "wasting away",
    "obesity": "excess body weight",
    "malnutrition": "poor nutrition",
    "dehydration": "lack of fluids",
    "hypoglycemia": "low blood sugar",
    "hyperglycemia": "high blood sugar",
    "diabetes mellitus": "high blood sugar condition",
    "insulin resistance": "body not responding to insulin",
    "hypothyroidism": "low thyroid activity",
    "hyperthyroidism": "overactive thyroid",
    "goiter": "swollen thyroid",
    "dysphagia": "difficulty swallowing",
    "dysphasia": "difficulty speaking",
    "aphasia": "loss of ability to speak",
    "ataxia": "loss of coordination",
    "paresthesia": "tingling or numbness",
    "paralysis": "loss of movement",
    "spasticity": "stiff muscles",
    "seizure": "uncontrolled brain activity",
    "epilepsy": "recurring seizures",
    "headache": "head pain",
    "migraine": "intense headache",
    "encephalopathy": "brain dysfunction",
    "meningitis": "brain lining infection",
    "delirium": "confused thinking",
    "dementia": "memory loss",
    "psychosis": "loss of reality",
    "mania": "extreme mood elevation",
    "depression": "persistent sadness",
    "anxiety": "excessive worry",
    "hallucination": "seeing or hearing things",
    "delusion": "false belief",
    "bipolar disorder": "mood swings",
    "schizophrenia": "chronic mental illness",
    "ptsd": "trauma-related stress",
    "ocd": "repetitive thoughts or actions",
    "insomnia": "difficulty sleeping",
    "narcolepsy": "excessive daytime sleepiness",
    "apnea": "stopping breathing",
    "arrhythmia": "irregular heart rhythm",
    "valvular disease": "heart valve problem",
    "congestive heart failure": "heart pumping problem",
    "cardiomyopathy": "heart muscle disease",
    "pericarditis": "heart lining inflammation",
    "angina": "chest pain",
    "aortic aneurysm": "bulge in the aorta",
    "deep vein thrombosis": "blood clot in leg vein",
    "pulmonary embolism": "clot in lung artery",
    "shock": "dangerously low blood pressure",
    "anaphylaxis": "severe allergic reaction",
    "urticaria": "hives",
    "eczema": "skin inflammation",
    "psoriasis": "scaly skin",
    "seborrheic dermatitis": "flaky scalp",
    "keratosis": "skin bump",
    "melanoma": "skin cancer",
    "basal cell carcinoma": "skin cancer",
    "squamous cell carcinoma": "skin cancer",
    "biopsy": "tissue sample",
    "pathology": "study of disease",
    "benign": "not cancerous",
    "malignant": "cancerous",
    "metastasis": "spread of cancer",
    "oncology": "cancer care",
    "radiotherapy": "radiation treatment",
    "chemotherapy": "cancer drug treatment",
    "immunotherapy": "immune-based treatment",
    "lymphadenopathy": "swollen lymph nodes",
    "tonsillitis": "tonsil infection",
    "sinusitis": "sinus infection",
    "otitis media": "ear infection",
    "conjunctivitis": "pink eye",
    "pharyngitis": "sore throat",
    "laryngitis": "voice box inflammation",
    "bronchiolitis": "small airway inflammation",
    "pneumothorax": "collapsed lung",
    "pleural effusion": "fluid around lungs",
    "atelectasis": "lung collapse",
    "intubation": "inserting breathing tube",
    "extubation": "removing breathing tube",
    "tracheostomy": "neck breathing tube",
    "ventilator": "breathing machine",
    "resuscitation": "reviving from death",
    "cardiac arrest": "heart stops",
    "do not resuscitate": "no revival order",
    "code blue": "medical emergency",
    "advance directive": "care instructions in advance",
    "palliative care": "comfort care",
    "hospice": "end-of-life care",
    "autopsy": "after-death exam",
    "morbidity": "illness",
    "mortality": "death",
    "prognosis": "expected outcome",
    "diagnosis": "identified condition",
    "treatment": "care plan",
    "prescription": "medicine order",
    "dosage": "medicine amount",
    "contraindication": "reason not to use",
    "adverse reaction": "bad effect",
    "side effect": "unintended effect",
    "tolerance": "resistance to drug",
    "dependence": "reliance on drug",
    "withdrawal": "symptoms after stopping",
    "overdose": "too much medicine",
    "toxicity": "poison effect",
    "placebo": "fake treatment",
    "clinical trial": "research study",
    "vital signs": "body measurements",
    "temperature": "body heat",
    "pulse": "heartbeat rate",
    "respiration": "breathing rate",
    "blood pressure": "pressure in arteries"
}


**RAG Code Implementaion**

In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

retriever_model = SentenceTransformer("all-MiniLM-L6-v2")

jargon_terms = list(medical_terms.keys())
layman_terms = list(medical_terms.values())

jargon_embeddings = retriever_model.encode(jargon_terms, convert_to_numpy=True)
index = faiss.IndexFlatL2(jargon_embeddings.shape[1])
index.add(jargon_embeddings)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
def simplify_summary(summary, top_k=1):
    words = summary.split()
    simplified = []

    for word in words:
        cleaned = word.strip(",.?!:;()").lower()
        if cleaned in medical_terms:
            simplified.append(medical_terms[cleaned])
            continue

        embedding = retriever_model.encode([cleaned], convert_to_numpy=True)
        D, I = index.search(embedding, top_k)

        if D[0][0] < 0.7:
            simplified.append(layman_terms[I[0][0]])
        else:
            simplified.append(word)

    return " ".join(simplified)


In [ ]:
for result in results[:5]:
    print("Original Summary:\n", result["combined"])
    simplified = simplify_summary(result["combined"])
    print("\nSimplified Summary:\n", simplified)
    print("=" * 60)


Original Summary:
 mr. ___ was admitted to the neurosurgery service after undergoing spinal stimulator placement. he tolerated the procedure well and was extubated in the operating room and transferred to the floor in stable condition. he tolerated the procedure well and was extubated in the operating room and transferred to the floor in stable condition. he was transferred to the floor in stable condition. on ___, the patient remained neurologically and hemodynamically stable. on ___, the patient remained neurologically intact and hemodynamically stable. he tolerated

Simplified Summary:
 mr. ___ was admitted to the neurosurgery service after undergoing spinal stimulator placement. he tolerated the procedure well and was removing breathing tube in the operating room and transferred to the floor in stable condition. he tolerated the procedure well and was removing breathing tube in the operating room and transferred to the floor in stable condition. he was transferred to the floor in s

**Example after implementing RAG**

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained("Vidit202/bart-mimic-summary")
tokenizer = AutoTokenizer.from_pretrained("Vidit202/bart-mimic-summary")
discharge_note = """Ms. ___ was admitted to the bariatric service with
abdominal pain after being transferred from an OSH with a CT
read of possible small bowel obstruction. Due to her ___ en y
gastric bypass, there was concern of an internal hernia and need
for operative intervention. On arrival, she had a nutritional
IV fluids given ("banana bag") which consisted of thiamine and
Vitamin B12. Stat CBC/chem10 and lactate revealed no etiology
of her abdominal pain. She had normal LFTs, lipase, lactate,
and white count. She was started on an IV BID PPI and IVF and
made NPO. She had a repeat CT abdomen with PO contrast to
better evaluate for a small bowel obstruction. There were no
abnormal findings on the CT scan. Her diet was advanced to
stage III which she tolerated well. Nutrition labs were drawn
which revealed iron deficiency. On questioning, she reported
not following up with a nutritionist and not being aware of
having her vitamin levels drawn by her PCP since her ___ en Y
gastric bypass. The importance of having close nutritional
follow up due to her altered anatomy was emphasized, including
following closely Vitamin B1, B12, iron, vitamin D, and folate.
Her primary care physician ___ was also telephoned and
a message was with left with his office to communicate these
recommendations. She had also been taking NSAIDs in the past
and was unaware of their danger with after a gastric bypass, and
the need to avoid NSAIDs was also reinforced.

On the day of discharge, she was tolerating a stage III
bariatric diet. Her pain was well controlled. She was voiding
freely. She was ambulating independently without assistance.
She will follow up with her PCP in one to two weeks."""

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/292 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

In [ ]:
input_only = f"Discharge Summary:\n{discharge_note}\n\nPlease simplify this discharge summary."
inputs = tokenizer(input_only, return_tensors="pt").to(model.device)
output = model.generate(**inputs, max_new_tokens=128)
summary_without_rag = tokenizer.decode(output[0], skip_special_tokens=True))



the patient was admitted to the bariatric surgery service on ___ for evaluation and treatment of possible small bowel obstruction. on admission, the patient was afebrile with stable vital signs; pain was well controlled oral pain medication on a prn basis. she was tolerating a stage IIIbariatric diet without nausea or vomiting. her vital signs were stable and she was voiding independently. her diet was advanced to a stage ii diet on ___. she was discharged home on ___ in stable condition with instructions to follow up with her primary care physician.
